In [40]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta as rd
import time
import math
from sklearn.metrics import mean_squared_error

In [41]:
# stocks data csv read
df = pd.read_csv('data.csv')
df = df.set_index('Date')

# s&p data csv read
df_sp = pd.read_csv('sp500.csv')
df_sp = df_sp.set_index('Date')

# stocks data csv read for partial replication
df_reduce = pd.read_csv('data.csv')
df_reduce = df_reduce.set_index('Date')

In [42]:
def date_slicer(df, start, duration, rebalancing_period=0):
    '''
    this function is used to slice out specific section of the data
    '''
    start = str(datetime.strptime(start, '%Y-%m-%d').date() + rd(months=rebalancing_period))
    end = str(datetime.strptime(start, '%Y-%m-%d').date() + rd(months=duration) - rd(days=1))
    return df.loc[start:end]

In [43]:
def data_process(df):
    '''
    this function gets the dataframe as input, processes it, and ouputs the cumulative change of the stocks
    that is used as input for training the model.
    '''
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    df = df.cumprod()
    df = df - 1
    df = df.iloc[-1,:]
    df = df.to_numpy()
    df = torch.from_numpy(df).type(torch.Tensor)
    return df

In [44]:
def daily_change(df):
    '''
    this function calculate the daily change of stocks included in the dataframe.
    '''
    df = df.pct_change()
    df = df.tail(-1)
    return df

In [45]:
def daily_return(df):
    '''
    this function calculate the daily return of stocks included in the dataframe, note that 
    daily return is equal to daily change + 1
    '''
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    return df

In [46]:
def index_finder(df):
    '''
    this function is just being used for extracting the stocks symbols
    '''
    df = df.pct_change()
    df = df.tail(-1)
    df = df + 1
    df = df.cumprod()
    df = df - 1
    df = df.iloc[-1,:]
    return df

In [47]:
# storing stocks symbols
stocks_index = index_finder(df).index

In [48]:
# shallow nnf biuld
class shallow_NNF(nn.Module):
    '''
    this class is used to train the data with Shallow NNF model, consisted of 2 fully connected layers, 
    a relU activation function in between and a softmax layer output that is translated into stock weights in portfolio.
    '''
    def __init__(self, input_dim, hidden_size, num_classes):
        super(shallow_NNF, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size) # fully connected layer
        self.fc2 = nn.Linear(hidden_size, num_classes) # fully connected layer
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.softmax(self.fc2(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [49]:
'''
shallow nnf partial biuld which is the same as original shallow nnf
this class helps us to use the full replication training to find the best companies to invest
and then find the optimal wieghts with the partial model
'''
class shallow_NNF_partial(nn.Module):
    def __init__(self, input_dim, hidden_size, num_classes):
        super(shallow_NNF_partial, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.softmax(self.fc2(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [50]:
# deep nnf build
class deep_NNF(nn.Module):
    '''
    this class is used to train the data with Deep NNF model, consisted of 6 fully connected layers, 
    relU activation functions in between and a softmax layer output that is translated into stock weights in portfolio.
    dropout is also included in deep NNF model.
    '''
    def __init__(self, input_dim, hidden_size1, hidden_size2, hidden_size3,
                 hidden_size4, hidden_size5, num_classes, dropout_p = 0.2):
        super(deep_NNF, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size1) # fully connected layer
        self.fc2 = nn.Linear(hidden_size1, hidden_size2) # fully connected layer
        self.fc3 = nn.Linear(hidden_size2, hidden_size3) # fully connected layer
        self.fc4 = nn.Linear(hidden_size3, hidden_size4) # fully connected layer
        self.fc5 = nn.Linear(hidden_size4, hidden_size5) # fully connected layer
        self.fc6 = nn.Linear(hidden_size5, num_classes) # fully connected layer
    
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_p)
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        self.fc3.reset_parameters()
        self.fc4.reset_parameters()
        self.fc5.reset_parameters()
        self.fc6.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.relu(self.fc3(out))
        out = self.dropout(out)
        out = self.relu(self.fc4(out))
        out = self.dropout(out)
        out = self.relu(self.fc5(out))
        out = self.softmax(self.fc6(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [51]:
'''
deep nnf partial biuld which is the same as original deep nnf
this class helps us to use the full replication training to find the best companies to invest
and then find the optimal wieghts with the partial model
'''
class deep_NNF_partial(nn.Module):
    def __init__(self, input_dim, hidden_size1, hidden_size2, hidden_size3,
                 hidden_size4, hidden_size5, num_classes, dropout_p = 0.2):
        super(deep_NNF_partial, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.fc5 = nn.Linear(hidden_size4, hidden_size5)
        self.fc6 = nn.Linear(hidden_size5, num_classes)
    
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_p)
        self.softmax = nn.Softmax(dim=0)
        
    def reset_parameters(self):
        self.fc1.reset_parameters()
        self.fc2.reset_parameters()
        self.fc3.reset_parameters()
        self.fc4.reset_parameters()
        self.fc5.reset_parameters()
        self.fc6.reset_parameters()
        
    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.relu(self.fc3(out))
        out = self.dropout(out)
        out = self.relu(self.fc4(out))
        out = self.dropout(out)
        out = self.relu(self.fc5(out))
        out = self.softmax(self.fc6(out))
        weights = out
        cumulative_change = sum(out * x)
        return cumulative_change, weights

In [52]:
# 1/N model build
class equal_w_model():
    '''
    this class is used to construct a portfolio with equal weights.
    '''
    def __init__(self, df):
        self.df = df
        self.performance()
        
    def performance(self):
        self.df = np.array(self.df)
        weights = np.ones((len(self.df), 1)) * (1/len(self.df))
        cumulative_change = sum(np.multiply(weights, self.df.reshape(-1,1)))
        return cumulative_change, weights.reshape(-1)

In [53]:
# rebalancing period = one or three months
rbp = 3

# number of companies in the partial portfolio
partial_num = 50

# epochs
num_epochs = 50

In [54]:
# shallow_nnf hyperparameters
input_dim = 471
hidden_size = 471
num_classes = 471
lr = 1e-3 # learning rate

In [55]:
# shallow nnf tune
'''
loss function is set to MSE and Adam optimizer is used in this model.
'''
shallow_NNF = shallow_NNF(input_dim=input_dim, hidden_size=hidden_size, num_classes=num_classes)
shallow_NNF_loss_fun = torch.nn.MSELoss(reduction='mean')
shallow_NNF_optimizer = torch.optim.Adam(shallow_NNF.parameters(), lr=lr)

In [56]:
# shallow nnf partial tune
'''
loss function is set to MSE and Adam optimizer is used in this model.
'''
shallow_NNF_partial = shallow_NNF_partial(input_dim=partial_num, hidden_size=hidden_size, num_classes=partial_num)
shallow_NNF_partial_loss_fun = torch.nn.MSELoss(reduction='mean')
shallow_NNF_partial_optimizer = torch.optim.Adam(shallow_NNF_partial.parameters(), lr=lr)

In [57]:
# deep_nnf hyperparameters
input_dim = 471
hidden_size1 = 471
hidden_size2 = 471
hidden_size3 = 471
hidden_size4 = 471
hidden_size5 = 471
num_classes = 471
lr = 1e-3 # learning rate
# probability of a neuron being shutdown that shuffles every epoch minimizing the overfit phenomenon
dropout_p = 0

In [58]:
# deep nnf tune
'''
like in shallow NNF, loss function is set to MSE and Adam optimizer is used.
'''
deep_NNF = deep_NNF(input_dim=input_dim, hidden_size1=hidden_size1, hidden_size2=hidden_size2, 
                    hidden_size3=hidden_size3, hidden_size4=hidden_size4, hidden_size5=hidden_size5,
                    num_classes=num_classes)
deep_NNF_loss_fun = torch.nn.MSELoss(reduction='mean')
deep_NNF_optimizer = torch.optim.Adam(deep_NNF.parameters(), lr=lr)

In [59]:
# deep nnf partial tune
'''
like in shallow NNF, loss function is set to MSE and Adam optimizer is used.
'''
deep_NNF_partial = deep_NNF_partial(input_dim=partial_num, hidden_size1=hidden_size1, hidden_size2=hidden_size2, 
                    hidden_size3=hidden_size3, hidden_size4=hidden_size4, hidden_size5=hidden_size5,
                    num_classes=partial_num)
deep_NNF_partial_loss_fun = torch.nn.MSELoss(reduction='mean')
deep_NNF_partial_optimizer = torch.optim.Adam(deep_NNF_partial.parameters(), lr=lr)

In [60]:
# RMSE
def RMSE(x, y, weights):
    '''
    this function calculates the root mean squere error of constructed portfollio and benchmark index 
    that is used for evaluating trained models.
    '''
    temp = 0
    for i in range(len(x)):
        temp += (sum(x.iloc[i] * weights) - y.iloc[i]) ** 2
    return math.sqrt(temp/len(x))

In [61]:
# MEAN
def MEAN(x, weights):
    '''
    this function calculates the mean return of the constructed portfolio during the given period.
    '''
    temp = []
    for i in range(len(x)):
        temp.append(sum(x.iloc[i] * weights))
    temp = np.array(temp)
    return temp.mean()

In [62]:
# Volatility
def VOL(x, weights):
    '''
    this function calculates the volatility of the constructed portfolio during the given period.
    '''
    temp = []
    for i in range(len(x)):
        temp.append(sum(x.iloc[i] * weights))
    temp = np.array(temp)
    return temp.std()

In [63]:
def portfolio_return(df, x_test, model, i, temp):   
    '''
    this function outputs the cumulative return of the portfolio test dataset of the given dataframe
    ''' 
    x_return = date_slicer(df, '2018-01-01', 1, i)
    x_return =  x_return.pct_change()
    x_return =  x_return.tail(-1)
    x_return =  x_return + 1
    x_return =  x_return.cumprod()
    
    if model == equal_w_model:
        weights = model(x_test).performance()[1]
    else:
        weights = np.array(model(x_test)[1].detach())
    
    for i in range(len(x_return)):
        temp.append(sum(x_return.iloc[i] * weights))
    temp = np.array(temp)
    return temp

In [64]:
def index_return(df_sp, i, temp):
    '''
    this function outputs the cumulative return of the benchmark index test dataset of the given dataframe
    '''
    y_return = date_slicer(df_sp, '2018-01-01', 1, i)
    y_return = y_return.pct_change()
    y_return = y_return.tail(-1)
    y_return = y_return + 1
    y_return = y_return.cumprod()
    
    for i in range(len(y_return)):
        temp.append(sum(y_return.iloc[i]))
    temp = np.array(temp)
    return temp

In [65]:
def valid_fun(x_valid, i, model):
    '''
    this function gets validation dataset, model and rebalaning period as input, then outputs the RMSE of given dataset.
    '''
    x_change = daily_change(date_slicer(df_reduce, '2017-07-01', 6, i))
    y_change = daily_change(date_slicer(df_sp, '2017-07-01', 6, i))
    # x_return = daily_return(date_slicer(df, '2017-07-01', 6, i))
    # y_return = daily_return(date_slicer(df_sp, '2017-07-01', 6, i))
    
    if model == equal_w_model:
        weights = model(x_valid).performance()[1]
    else:
        weights = np.array(model(x_valid)[1].detach())
    
    valid_rmse = RMSE(x_change, y_change, weights)
    # valid_mean = MEAN(x_return, weights)
    # valid_vol  = VOL(x_return, weights)
    
    print(f'Validation RMSE: {valid_rmse}')
    # print(f'Validation MEAN: {valid_mean}')
    # print(f'Validation VOL: {valid_vol}')
    
    return valid_rmse

In [66]:
def test_fun(x_test, i, model):
    '''
    this function gets test dataset, model and rebalaning period as input, then outputs the RMSE, Mean and volatility 
    of the given dataset.
    '''
    x_change = daily_change(date_slicer(df_reduce, '2018-01-01', 6, i))
    y_change = daily_change(date_slicer(df_sp, '2018-01-01', 6, i))
    x_return = daily_return(date_slicer(df_reduce, '2018-01-01', 6, i))
    y_return = daily_return(date_slicer(df_sp, '2018-01-01', 6, i))
    
    if model == equal_w_model:
        weights = model(x_test).performance()[1]
    else:
        weights = np.array(model(x_test)[1].detach())
    
    test_rmse = RMSE(x_change, y_change, weights)
    test_mean = MEAN(x_return, weights)
    test_vol  = VOL(x_return, weights)
    test_dic = {'RMSE': test_rmse, 'MEAN': test_mean, 'VOL': test_vol}
    
    print(f'Test RMSE: {test_rmse}')
    print(f'Test MEAN: {test_mean}')
    print(f'Test VOL: {test_vol}')
    
    return test_dic

### **Deep NNF Training**

In [67]:
# deep nnf training function
'''
this function is used to train the model using x_train & y_train given to it, printing MSE of trained model in first and last
 epich and also printing train time of the model
'''
def train_deep_nnf(x_train, y_train, i):
    start_time_deep_nnf = time.time()
    print(f'\nDeep NNF Training & Results for model {(i/rbp)+1} (Full replication) :')
    
    for epoch in range(num_epochs):
        y_train_pred = deep_NNF(x_train)[0]
        loss_deep_nnf = deep_NNF_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            weights = np.array(deep_NNF(x_train)[1].detach())
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_deep_nnf.item()}')
        deep_NNF_optimizer.zero_grad()
        loss_deep_nnf.backward()
        deep_NNF_optimizer.step()
        
    training_time = format(time.time()-start_time_deep_nnf, '0.2f')
    print(f'Training time: {training_time}')
    
    return weights

In [68]:
# deep nnf partial training function
def train_deep_nnf_partial(x_train, y_train, i):    
    start_time_deep_nnf = time.time()
    print(f'\nDeep NNF Training & Results for model {(i/rbp)+1} (Partial replication):')
    
    for epoch in range(num_epochs):
        y_train_pred = deep_NNF_partial(x_train)[0]
        loss_deep_nnf = deep_NNF_partial_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_deep_nnf.item()}')
        deep_NNF_partial_optimizer.zero_grad()
        loss_deep_nnf.backward()
        deep_NNF_partial_optimizer.step()
        
    training_time = format(time.time()-start_time_deep_nnf, '0.2f')
    print(f'Training time: {training_time}')

In [69]:
def partial(x_train, x_valid, x_test, weights, stocks_index, num = partial_num):
    df_partial = pd.DataFrame({'x_train': x_train, 'x_valid': x_valid, 'x_test': x_test,
                               'weights': weights}, index = stocks_index)
    df_partial = df_partial.sort_values(by = ['weights'])
    out_index = df_partial.index[num:]
    df_partial = df_partial.iloc[:num]
    
    x_train = df_partial['x_train'].to_numpy()
    x_valid = df_partial['x_valid'].to_numpy()
    x_test = df_partial['x_test'].to_numpy()
    
    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    x_valid = torch.from_numpy(x_valid).type(torch.Tensor)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    
    return x_train, x_valid, x_test, out_index

In [70]:
# deep nnf
'''
in this cell,firstly, train, validation and test datasets are sliced in each loop. then deep NNf outputs the best stocks with 
full replication and then we use the specific stocks to train the model again and get the optimal weights (each loop)
then best model will be chosen. Also RMSE, Mean and volatility of all models and then the best model is printed.
'''
deep_nnf_valid_rmse_list = []
deep_nnf_test_results = []
out_index_history = []
deep_nnf_test_plot = [] # storing the deep model test data return for plotting later on
index_test_plot = [] # storing the index test data return for plotting later on

for i in range(int(24/rbp)):
    df_reduce = df.copy()    
    x_train = data_process(date_slicer(df, '2014-07-01', 36, i*rbp))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i*rbp))
    x_valid = data_process(date_slicer(df, '2017-07-01', 6, i*rbp))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i*rbp))
    x_test = data_process(date_slicer(df, '2018-01-01', 1, i*rbp))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i*rbp))
    weights = train_deep_nnf(x_train, y_train, i*rbp)
    x_train, x_valid, x_test, out_index = partial(x_train, x_valid, x_test, weights, stocks_index, num = partial_num)
    out_index_history.append(out_index)
    df_reduce = df_reduce.drop(out_index, axis=1)
    train_deep_nnf_partial(x_train, y_train, i*rbp)
    deep_nnf_valid_rmse_list.append(valid_fun(x_valid, i*rbp, deep_NNF_partial))
    deep_nnf_test_results.append(test_fun(x_test, i*rbp, deep_NNF_partial))
    portfolio_return(df_reduce, x_test, deep_NNF_partial, i, deep_nnf_test_plot)
    index_return(df_sp, i, index_test_plot)
    deep_NNF.reset_parameters()
    deep_NNF_partial.reset_parameters()

print(f'\nMin Valid RMSE is: {min(deep_nnf_valid_rmse_list)} for model i = {deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))+1}')
print('Selected Model Test Results are:')
print('RMSE =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['RMSE'])
print('MEAN =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['MEAN'])
print('VOL =', deep_nnf_test_results[deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))]['VOL'])

deep_best_result_index = deep_nnf_valid_rmse_list.index(min(deep_nnf_valid_rmse_list))
deep_nnf_test_plot = np.array(deep_nnf_test_plot).reshape(-1,1)
index_test_plot = np.array(index_test_plot).reshape(-1,1)


Deep NNF Training & Results for model 1.0 (Full replication) :
Epoch 1 of 10 | MSE: 0.04576672986149788
Epoch 10 of 10 | MSE: 2.7898104235646315e-05
Training time: 0.11

Deep NNF Training & Results for model 1.0 (Partial replication):
Epoch 1 of 10 | MSE: 1.1630672216415405
Epoch 10 of 10 | MSE: 0.11094827950000763
Training time: 0.05
Validation RMSE: 0.002603573623611535
Test RMSE: 0.002783187123030721
Test MEAN: 1.0007080186151753
Test VOL: 0.009928726956642712


/Users/hamidrezarahimzadeh/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning:

Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.




Deep NNF Training & Results for model 2.0 (Full replication) :
Epoch 1 of 10 | MSE: 0.060359105467796326
Epoch 10 of 10 | MSE: 0.00017864475375972688
Training time: 0.10

Deep NNF Training & Results for model 2.0 (Partial replication):
Epoch 1 of 10 | MSE: 1.5597609281539917
Epoch 10 of 10 | MSE: 0.24643553793430328
Training time: 0.04
Validation RMSE: 0.00406127756807631
Test RMSE: 0.003562123989246157
Test MEAN: 1.0012533749186343
Test VOL: 0.007433502033991717

Deep NNF Training & Results for model 3.0 (Full replication) :
Epoch 1 of 10 | MSE: 0.039711933583021164
Epoch 10 of 10 | MSE: 0.00988954771310091
Training time: 0.10

Deep NNF Training & Results for model 3.0 (Partial replication):
Epoch 1 of 10 | MSE: 1.6051511764526367
Epoch 10 of 10 | MSE: 0.18667708337306976
Training time: 0.05
Validation RMSE: 0.0036003180298986677
Test RMSE: 0.005051869300410573
Test MEAN: 0.9989478605606705
Test VOL: 0.013719933597843207

Deep NNF Training & Results for model 4.0 (Full replication) :

### **Shallow NNF Training**

In [71]:
# shallow nnf training function
def train_shallow_nnf(x_train, y_train, i):
    '''
    this function is used to train the model using x_train & y_train given to it, printing MSE of trained model in first and last
    epoch and also printing train time of the model
    '''
    start_time_shallow_nnf = time.time()
    print(f'\nShallow NNF Training & Results for model {(i/rbp)+1}:')
    
    for epoch in range(num_epochs):
        y_train_pred = shallow_NNF(x_train)[0]
        loss_shallow_nnf = shallow_NNF_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            weights = np.array(deep_NNF(x_train)[1].detach())
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_shallow_nnf.item()}')
        shallow_NNF_optimizer.zero_grad()
        loss_shallow_nnf.backward()
        shallow_NNF_optimizer.step()
        
    training_time = format(time.time()-start_time_shallow_nnf, '0.2f')
    print(f'Training time: {training_time}')
    
    return weights

In [72]:
# shallow nnf partial training function
def train_shallow_nnf_partial(x_train, y_train, i):    
    start_time_shallow_nnf = time.time()
    print(f'\nDeep NNF Training & Results for model {(i/rbp)+1} (Partial replication):')
    
    for epoch in range(num_epochs):
        y_train_pred = shallow_NNF_partial(x_train)[0]
        loss_shallow_nnf = shallow_NNF_partial_loss_fun(y_train_pred, y_train)
        if epoch == 0 or epoch == num_epochs-1:
            print(f'Epoch {epoch+1} of {num_epochs} | MSE: {loss_shallow_nnf.item()}')
        shallow_NNF_partial_optimizer.zero_grad()
        loss_shallow_nnf.backward()
        shallow_NNF_partial_optimizer.step()
        
    training_time = format(time.time()-start_time_shallow_nnf, '0.2f')
    print(f'Training time: {training_time}')

In [73]:
#shallow nnf
'''
in this cell,firstly, train, validation and test datasets are sliced in each loop. then shallow NNf outputs the best stocks with 
full replication and then we use the specific stocks to train the model again and get the optimal weights (each loop)
then best model will be chosen. Also RMSE, Mean and volatility of all models and then the best model is printed.
'''
shallow_nnf_valid_rmse_list = []
shallow_nnf_test_results = []
shallow_nnf_test_plot = [] # storing the shallow model test data return for plotting later on

for i in range(int(24/rbp)):
    df_reduce = df.copy()
    x_train = data_process(date_slicer(df, '2014-07-01', 36, i*rbp))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i*rbp))
    x_valid = data_process(date_slicer(df, '2017-07-01', 6, i*rbp))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i*rbp))
    x_test = data_process(date_slicer(df, '2018-01-01', 1, i*rbp))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i*rbp))
    weights = train_shallow_nnf(x_train, y_train, i*rbp)
    x_train, x_valid, x_test, out_index = partial(x_train, x_valid, x_test, weights, stocks_index, num = partial_num)
    df_reduce = df_reduce.drop(out_index, axis=1)
    train_shallow_nnf_partial(x_train, y_train, i*rbp)
    shallow_nnf_valid_rmse_list.append(valid_fun(x_valid, i*rbp, shallow_NNF_partial))
    shallow_nnf_test_results.append(test_fun(x_test, i*rbp, shallow_NNF_partial))
    portfolio_return(df_reduce, x_test, shallow_NNF_partial, i, shallow_nnf_test_plot)
    shallow_NNF.reset_parameters()
    shallow_NNF_partial.reset_parameters()

# print(f'\nMin Valid RMSE is: {min(valid_rmse_list)} for model i = {(deep_best_result_index)+1}')
print('Selected Model Test Results for model i =', (deep_best_result_index)+1, 'are: ')
print('RMSE =', shallow_nnf_test_results[(deep_best_result_index)]['RMSE'])
print('MEAN =', shallow_nnf_test_results[(deep_best_result_index)]['MEAN'])
print('VOL =', shallow_nnf_test_results[(deep_best_result_index)]['VOL'])

shallow_nnf_test_plot = np.array(shallow_nnf_test_plot).reshape(-1,1)


Shallow NNF Training & Results for model 1.0:
Epoch 1 of 10 | MSE: 0.047189950942993164
Epoch 10 of 10 | MSE: 0.0008117908728308976
Training time: 0.07

Deep NNF Training & Results for model 1.0 (Partial replication):
Epoch 1 of 10 | MSE: 0.10243089497089386
Epoch 10 of 10 | MSE: 0.0011636066483333707
Training time: 0.01
Validation RMSE: 0.001856581934988317
Test RMSE: 0.002208954769116824
Test MEAN: 1.000093617340787
Test VOL: 0.009614344629706147

Shallow NNF Training & Results for model 2.0:
Epoch 1 of 10 | MSE: 0.05882757529616356
Epoch 10 of 10 | MSE: 0.007734063547104597
Training time: 0.06

Deep NNF Training & Results for model 2.0 (Partial replication):
Epoch 1 of 10 | MSE: 0.16543707251548767
Epoch 10 of 10 | MSE: 0.00021248288976494223
Training time: 0.01
Validation RMSE: 0.0022024651628990556
Test RMSE: 0.002047568063379345
Test MEAN: 1.000875751731011
Test VOL: 0.005789363090760106

Shallow NNF Training & Results for model 3.0:
Epoch 1 of 10 | MSE: 0.04022863134741783
Epoc

### **1/N Model**

In [74]:
'''
here we run the 1/N model, for the number of stocks, each stock gets the weight of 1/N meaning that
every stock is equally important, this model play the role of a benchmark to see how effective our model are
'''
equal_w_model_valid_rmse_list = []
equal_w_model_test_results = []
equal_w_model_test_plot = [] # storing the 1/n model test data return for plotting later on

for i in range(int(24/rbp)):
    df_reduce = df.copy()
    df_reduce = df_reduce.drop(out_index_history[i], axis=1)
    print(f'\nEqual Weights Model Results for model {i+1}:')
    x_train = data_process(date_slicer(df_reduce, '2014-07-01', 36, i*rbp))
    y_train = data_process(date_slicer(df_sp, '2014-07-01', 36, i*rbp))
    x_valid = data_process(date_slicer(df_reduce, '2017-07-01', 6, i*rbp))
    y_valid = data_process(date_slicer(df_sp, '2017-07-01', 6, i*rbp))
    x_test = data_process(date_slicer(df_reduce, '2018-01-01', 1, i*rbp))
    y_test = data_process(date_slicer(df_sp, '2018-01-01', 1, i*rbp))
    
    equal_w_model_valid_rmse_list.append(valid_fun(x_valid, i*rbp, equal_w_model))
    equal_w_model_test_results.append(test_fun(x_test, i*rbp, equal_w_model))
    portfolio_return(df_reduce, x_test, equal_w_model, i, equal_w_model_test_plot)
    
print('Selected Model Test Results for model i =', (deep_best_result_index)+1, 'are: ')
print('RMSE =', equal_w_model_test_results[(deep_best_result_index)]['RMSE'])
print('MEAN =', equal_w_model_test_results[(deep_best_result_index)]['MEAN'])
print('VOL =', equal_w_model_test_results[(deep_best_result_index)]['VOL'])

equal_w_model_test_plot = np.array(equal_w_model_test_plot).reshape(-1,1)


Equal Weights Model Results for model 1:
Validation RMSE: 0.0028697472289477464
Test RMSE: 0.002612447787867624
Test MEAN: 1.0007578565699635
Test VOL: 0.010065460979996322

Equal Weights Model Results for model 2:
Validation RMSE: 0.004159345733954496
Test RMSE: 0.0035677375056315363
Test MEAN: 1.0013003988873346
Test VOL: 0.0076340066343048435

Equal Weights Model Results for model 3:
Validation RMSE: 0.0038568232465743904
Test RMSE: 0.005513872288860587
Test MEAN: 0.9988827842977956
Test VOL: 0.014238280000508004

Equal Weights Model Results for model 4:
Validation RMSE: 0.004396383724639134
Test RMSE: 0.006285642911346925
Test MEAN: 1.0000178515818765
Test VOL: 0.01657948131764014

Equal Weights Model Results for model 5:
Validation RMSE: 0.005414594792879423
Test RMSE: 0.0035676176794984314
Test MEAN: 1.0021253586882646
Test VOL: 0.01006587638929778

Equal Weights Model Results for model 6:
Validation RMSE: 0.006119391602061266
Test RMSE: 0.003870030332358839
Test MEAN: 1.0003216

In [75]:
# print test results
'''
here we compare the results in a dataframe featuring RMSE, MEAN and, volatility of each model in the test dataset
that has the best results for deep nnf model. this dataframe can cope with the understanding of why we bother 
implementing a complex neural network
'''
print(f'Models test results with rebalancing period of {rbp} month(s) are: ')
deep_temp = pd.DataFrame(deep_nnf_test_results)
deep_temp = deep_temp.iloc[deep_best_result_index]
shallow_temp = pd.DataFrame(shallow_nnf_test_results)
shallow_temp = shallow_temp.iloc[deep_best_result_index]
equal_w_temp = pd.DataFrame(equal_w_model_test_results)
equal_w_temp = equal_w_temp.iloc[deep_best_result_index]

# extract the mean and volatility of the s&p index on the test dataset
sp_temp_rmse = '-'
sp_temp_mean = daily_return(date_slicer(df_sp, '2018-01-01', 6, deep_best_result_index)).mean()[0]
sp_temp_std = daily_return(date_slicer(df_sp, '2018-01-01', 6, deep_best_result_index)).std()[0]
sp_temp = pd.DataFrame([sp_temp_rmse, sp_temp_mean, sp_temp_std], index=deep_temp.index)

# concatinating the result in a unified dataframe
final_result = pd.concat([deep_temp, shallow_temp, equal_w_temp, sp_temp], axis=1, join='inner')
final_result.columns = ['Deep NNF', 'Shallow NNF', '1/N Model', 'S&P 500']
final_result

Models test results with rebalancing period of 3 month(s) are: 


,Deep NNF,Shallow NNF,1/N Model,S&P 500
RMSE,0.002783,0.002209,0.002612,-
MEAN,1.000708,1.000094,1.000758,1.000121
VOL,0.009929,0.009614,0.010065,0.010367


In [76]:
'''
to further showcase the results, here we compute the average RMSE of each model in test dataset
'''
print(f'Average of test RMSE for each model: ')

deep_nnf_test_rmse_mean = 0 # temp variable for storing each tmse for deep nnf model
for i in range(int(24/rbp)):
    deep_nnf_test_rmse_mean += deep_nnf_test_results[i]['RMSE']
print(f'Deep NNF: {deep_nnf_test_rmse_mean/int(24/rbp)}')

shallow_nnf_test_rmse_mean = 0 # temp variable for storing each tmse for shallow nnf model
for i in range(int(24/rbp)):
    shallow_nnf_test_rmse_mean += shallow_nnf_test_results[i]['RMSE']
print(f'Shallow NNF: {shallow_nnf_test_rmse_mean/int(24/rbp)}')

equal_w_model_test_rmse_mean = 0 # temp variable for storing each tmse for 1/n model model
for i in range(int(24/rbp)):
    equal_w_model_test_rmse_mean += equal_w_model_test_results[i]['RMSE']
print(f'Equal weight model: {equal_w_model_test_rmse_mean/int(24/rbp)}')

Average of test RMSE for each model: 
Deep NNF: 0.004041545576902323
Shallow NNF: 0.002351085977528434
Equal weight model: 0.003934138966890342


In [77]:
# concatinating the test dataset return results of each model + index return for plot
plot_test = pd.concat([pd.DataFrame(deep_nnf_test_plot), pd.DataFrame(shallow_nnf_test_plot),
                       pd.DataFrame(equal_w_model_test_plot), pd.DataFrame(index_test_plot)], axis=1, join='inner')
plot_test.columns = ['Deep NNF', 'Shallow NNF', '1/N Model', 'S&P 500']

In [78]:
# importing a module for better and more interactive plot
import cufflinks as cf
cf.set_config_file(offline = True)

'''
plotting deep nnf, shallow nnf and, 1/n model performance on the test dataset, compare them with
index (s&p) for better understanding
'''
plot_test.iplot()